In [95]:
import geopandas as gpd
import pandas as pd

## Open Counties data

In [96]:
file_path = "data/cb_2024_us_county_500k/cb_2024_us_county_500k.shp"
counties_gdf = gpd.read_file(file_path)
counties_gdf = counties_gdf.rename(columns={"GEOIDFQ": "AFFGEOID"})

## Get Tennessee 2023 Totals

In [97]:
population_2023_df = pd.read_csv(
    "data/ACSDP5Y2023.DP05_2025-06-20T013737/ACSDP5Y2023.DP05-Data.csv",
    low_memory=False,
)

In [98]:
population_2023_df.columns = population_2023_df.iloc[0]
population_2023_df = population_2023_df[1:]
population_2023_df = population_2023_df[
    ["Geography", "Geographic Area Name", "Estimate!!SEX AND AGE!!Total population"]
]

## Get Tennessee 2000 Totals

In [99]:
population_2000_df = pd.read_csv(
    "data/DECENNIALDPSF22000.DP1_2025-06-20T014335/DECENNIALDPSF22000.DP1-Data.csv",
    low_memory=False,
)

In [100]:
population_2000_df.columns = population_2000_df.iloc[0]
population_2000_df = population_2000_df[1:]
population_2000_df = population_2000_df[["Geography", "Number!!Total population"]]

## Merge Data

In [101]:
population_totals_df = population_2000_df.merge(
    population_2023_df, on="Geography", how="inner"
)
population_totals_df = population_totals_df.rename(
    columns={
        "Geography": "AFFGEOID",
        "Number!!Total population": "2000",
        "Estimate!!SEX AND AGE!!Total population": "2023",
    }
)

In [102]:
population_totals_df[["2000", "2023"]] = population_totals_df[["2000", "2023"]].astype(
    int
)
population_totals_df["TOTAL"] = (
    population_totals_df["2023"] - population_totals_df["2000"]
)
population_totals_df["PERCENT"] = (
    (population_totals_df["TOTAL"] / population_totals_df["2000"]) * 100
).round(decimals=2)
population_totals_df = population_totals_df.drop_duplicates("AFFGEOID").reset_index(
    drop=True
)

In [103]:
diff_gdf = counties_gdf.merge(population_totals_df, on="AFFGEOID", how="inner")

In [104]:
diff_gdf["county_id"] = diff_gdf["AFFGEOID"].str[9:]
diff_gdf["state_id"] = diff_gdf["AFFGEOID"].str[9:11]

In [105]:
diff_gdf = diff_gdf.to_crs(9311)
diff_gdf[diff_gdf["STUSPS"] == "TN"].to_file("data/population_diff_tn.gpkg")

In [110]:
diff_gdf[diff_gdf["STUSPS"] == "TN"]["2023"].sum()

np.int64(6986082)

In [111]:
diff_gdf[diff_gdf["STUSPS"] == "TN"]["2000"].sum()

np.int64(5689283)

In [106]:
diff_gdf[diff_gdf["STUSPS"] == "TN"].sort_values("PERCENT", ascending=False)

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,ALAND,AWATER,geometry,2000,Geographic Area Name,2023,TOTAL,PERCENT,county_id,state_id
887,47,187,01639801,0500000US47187,47187,Williamson,Williamson County,TN,Tennessee,06,1509732497,3011572,"POLYGON ((1150522.818 -926919.961, 1150453.334...",126638,"Williamson County, Tennessee",254609,127971,101.05,47187,47
1085,47,149,01639787,0500000US47149,47149,Rutherford,Rutherford County,TN,Tennessee,06,1603892843,12406556,"POLYGON ((1198429.041 -935700.545, 1198424.387...",182023,"Rutherford County, Tennessee",351591,169568,93.16,47149,47
234,47,189,01639802,0500000US47189,47189,Wilson,Wilson County,TN,Tennessee,06,1479109875,29878235,"POLYGON ((1199461.202 -877063.869, 1199570.207...",88809,"Wilson County, Tennessee",153587,64778,72.94,47189,47
1967,47,125,01639777,0500000US47125,47125,Montgomery,Montgomery County,TN,Tennessee,06,1396441773,12068782,"POLYGON ((1100755.163 -847803.851, 1107466.258...",134768,"Montgomery County, Tennessee",227957,93189,69.15,47125,47
3158,47,169,01648581,0500000US47169,47169,Trousdale,Trousdale County,TN,Tennessee,06,296143586,5960709,"POLYGON ((1224989.253 -860312.312, 1225064.375...",7259,"Trousdale County, Tennessee",11805,4546,62.63,47169,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1711,47,131,01639779,0500000US47131,47131,Obion,Obion County,TN,Tennessee,06,1411149170,27448111,"POLYGON ((942939.761 -916250.171, 944112.871 -...",32450,"Obion County, Tennessee",30570,-1880,-5.79,47131,47
1101,47,097,01639763,0500000US47097,47097,Lauderdale,Lauderdale County,TN,Tennessee,06,1222203226,80251266,"POLYGON ((906800.844 -975996.859, 907157.667 -...",27101,"Lauderdale County, Tennessee",24933,-2168,-8.00,47097,47
2675,47,069,01639750,0500000US47069,47069,Hardeman,Hardeman County,TN,Tennessee,06,1729509835,6768751,"POLYGON ((984257.307 -1046316.303, 984126.782 ...",28105,"Hardeman County, Tennessee",25477,-2628,-9.35,47069,47
2244,47,075,01639753,0500000US47075,47075,Haywood,Haywood County,TN,Tennessee,06,1380741525,2472509,"POLYGON ((949626.989 -986935.184, 949994.798 -...",19797,"Haywood County, Tennessee",17672,-2125,-10.73,47075,47
